This notebook provides code for PRG classification

In [ ]:
import torch
from torch import nn
import numpy as np
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!unzip "/content/drive/MyDrive/nets/prg_classification/prg_data.zip" -d "/content" &> /dev/null#   ---> Train data
#!unzip "/content/drive/MyDrive/nets/prg_classification/whole_candidates.zip" -d "/content" &> /dev/null

In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

## Part 1 - Data Preprocessing

### Visualising data and creating DataLoaders

In [ ]:
from pathlib import Path
from PIL import Image

# Setup path to data folder
image_path = Path("/content/")
if image_path.is_dir():
  print(f"{image_path} directory exists.")

'''
Initialising train, validation and train paths.

prg_path = image_path / "PRG"
others_path = image_path / "others"
prg_path_list = list(prg_path.glob("*.*"))
others_path_list = list(others_path.glob("*.*"))
print(len(prg_path_list), len(others_path_list))

PRG:71
Others:943
Split: 55/6/10
       750/100/93
'''

train_dir = image_path / "augmented_train"
#train_dir = image_path / "small_train"
valid_dir = image_path / "validation"
#valid_dir = image_path / "extended_validation" #(test + prg from validation)
#valid_dir = image_path / "small_validation"
#test_dir = image_path / "test"
#test_dir = image_path / "small_test"
test_dir = image_path / "augmented_test"

## Segmentation step by step

In [ ]:
import random

#random.seed(42)

image_path_list = list(train_dir.glob("*/*.jpg"))

'''
Let's visualise random image
'''
random_image_path = random.choice(image_path_list)
image_class = random_image_path.parent.stem
original_img = Image.open(random_image_path)
original_img

In [ ]:
!pip install astropy &> /dev/null
!pip install photutils &> /dev/null

In [ ]:
import matplotlib.pyplot as plt
img = original_img.convert("L")
#img = img[:,:,0] --> For different color channels
img = np.array(img)
img = img.astype("float64")
plt.imshow(img)

In [ ]:
from photutils.background import Background2D, MedianBackground
import cv2

bkg_estimator = MedianBackground()
bkg = Background2D(img, (10, 10), filter_size=(3, 3),
                   bkg_estimator=bkg_estimator)
img -= bkg.background  # subtract the background
plt.imshow(img)

In [ ]:
from astropy.convolution import convolve
from photutils.segmentation import make_2dgaussian_kernel
threshold = 1.5 * bkg.background_rms
kernel = make_2dgaussian_kernel(3.0, size=5)  # FWHM = 3.0
convolved_data = convolve(img, kernel)
plt.imshow(convolved_data)

In [ ]:
from photutils.segmentation import detect_sources
segment_map = detect_sources(convolved_data, threshold, npixels=10)
print(segment_map)

In [ ]:
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
norm = ImageNormalize(stretch=SqrtStretch())
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12.5))
ax1.imshow(img, origin='lower', cmap='Greys_r', norm=norm)
ax1.set_title('Background-subtracted Data')
ax2.imshow(segment_map, origin='lower', cmap=segment_map.cmap,
           interpolation='nearest')
ax2.set_title('Segmentation Image')

In [ ]:
from photutils.segmentation import deblend_sources
segm_deblend = deblend_sources(convolved_data, segment_map,
                               npixels=10, nlevels=32, contrast=0.001,
                               progress_bar=False)
plt.imshow(segm_deblend, origin='lower', cmap=segment_map.cmap,
           interpolation='nearest')

In [ ]:
print(segm_deblend.areas)
print(np.argmax(segm_deblend.areas))
print(segm_deblend.labels)
print(segm_deblend.labels[np.argmax(segm_deblend.areas)])
print(segm_deblend.get_area(segm_deblend.labels[np.argmax(segm_deblend.areas)]))
print(segm_deblend.max_label)
print(segm_deblend.slices[12])

In [ ]:
center = np.array((segm_deblend.data.shape[0] // 2, segm_deblend.data.shape[1] // 2))
candidates = []
for n, slc in enumerate(segm_deblend.slices):
  x, y = slc[0], slc[1]
  slc_center = np.array([(x.start + x.stop)//2, (y.start + y.stop)//2])
  metric = ((slc_center[0] - center[0])**2 + (slc_center[1] - center[1])**2)**(1/2)
  if metric < 0.1*segm_deblend.data.shape[1]:
    candidates.append(n)

In [ ]:
#slc = segm_deblend.slices[np.argmax(segm_deblend.areas)] --> max not suitable cause it is possible to have another bright source on the image
slc = segm_deblend.slices[candidates[0]]
plt.imshow(segm_deblend[slc], origin='lower', cmap=segment_map.cmap,
           interpolation='nearest')

In [ ]:
#mask = np.array(segment_map.data == segment_map.labels[np.argmax(segment_map.areas)])
#debl_mask = np.array(segm_deblend.data == segm_deblend.labels[np.argmax(segm_deblend.areas)])
mask = np.array(segment_map.data == segment_map.labels[candidates[0]])
debl_mask = np.array(segm_deblend.data == segm_deblend.labels[candidates[0]])
#plt.imshow(segm_deblend.data*mask, origin='lower', cmap=segment_map.cmap,
#           interpolation='nearest')

#fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(1, 6, figsize=(16, 16))
fig, (ax1, ax2, ax3, ax5) = plt.subplots(1, 4, figsize=(16, 16))

ax1.imshow(original_img)
ax1.set_title('Original image')

ax2.imshow(img, origin='lower', cmap='Greys_r', norm=norm)
ax2.set_title('Background-subtracted Data')

ax3.imshow(img*mask, origin='lower', cmap='Greys_r', norm=norm)
ax3.set_title('Mask')

#ax4.imshow(img*debl_mask, origin='lower', cmap='Greys_r', norm=norm)
#ax4.set_title('Deblended mask')

ax5.imshow(original_img*np.repeat(mask[:, :, np.newaxis], 3, axis=2))#casting 2D grayscale mask to 3D RGB mask repeating it 3 times, can check like [:,:,0], [:,:,1], [:,:,2]
ax5.set_title('Segmented image')

#ax6.imshow(original_img*np.repeat(debl_mask[:, :, np.newaxis], 3, axis=2))#casting 2D grayscale mask to 3D RGB mask repeating it 3 times, can check like [:,:,0], [:,:,1], [:,:,2]
#ax6.set_title('Deblended original img')
plt.tight_layout()
plt.savefig("segmap_shoowcase.pdf", dpi=400, format='pdf', bbox_inches='tight')

### Let's visualise our data

In [ ]:
import random

random.seed(42)

image_path_list = list(image_path.glob("*/*/*.jpg"))

'''
Let's visualise random image
'''
random_image_path = random.choice(image_path_list)

# The image class is the name of the directory where the image is stored
image_class = random_image_path.parent.stem
img = Image.open(random_image_path)
print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img

### Also let's generate additional prg galaxies using augmentations

In [ ]:
from torchvision import transforms
import matplotlib.pyplot as plt

test_prg_path = test_dir / "prg"
test_prg_list = list(test_prg_path.glob("*"))
print(len(test_prg_list))

transform = transforms.Compose([
  transforms.RandomHorizontalFlip(p=0.8), # p = probability of flip, 0.5 = 50% chance
  transforms.RandomVerticalFlip(p=0.8),
  transforms.RandomRotation(degrees=(0, 360))
])

In [ ]:
train_prg_path = train_dir / "prg"
train_prg_list = list(train_prg_path.glob("*"))

In [ ]:
NUM_OF_AUGM = 8

for image_path in test_prg_list:
  with Image.open(image_path) as f:
    for i in range(NUM_OF_AUGM):
      transformed_img = transform(f)
      transformed_img.save(str(image_path)[:81] + f"aug{i+1}_" + str(image_path)[81:])

## Creating dataloaders

In [ ]:
from torch.utils.data import DataLoader
from typing import Tuple, List, Dict

In [ ]:
# Make function to find classes in target directory
def find_classes(directory: str) -> Tuple[List[str], Dict[str, int]]:
  import os
  """
  Finds the class folder names in a target directory.

  Assumes target directory is in standard image classification format.

  Args:
    directory (str): target directory to load classnames from.

  Returns:
    Tuple[List[str], Dict[str, int]]: (list_of_class_names, dict(class_name: idx...))

  """
  # 1. Get the class names by scanning the target directory
  classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
  if not classes:
    raise FileNotFoundError(f"Couldn't find any classes in {directory}.")

  class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
  return classes, class_to_idx

### Creating custom Dataset

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
  def __init__(self, targ_dir: str, transform=None) -> None:
    self.paths = list(Path(targ_dir).glob("*/*"))
    self.transform = transform
    self.classes, self.class_to_idx = find_classes(targ_dir)

  def load_image(self, index: int) -> Image.Image:
    '''
    Opens an image via a path and returns it.
    '''
    image_path = self.paths[index]
    return Image.open(image_path)

  def __len__(self) -> int:
    '''
    Returns the total number of samples.
    '''
    return len(self.paths)

  def __getitem__(self, index: int) -> Tuple[Image.Image, int]:#or Tuple[torch.Tensor, int] with appropriate transform
    '''
    Returns one sample of data, data and label (X, y).
    '''
    img = self.load_image(index)
    class_name  = self.paths[index].parent.name
    class_idx = self.class_to_idx[class_name]

    # Transform if necessary
    if self.transform:
      return self.transform(img), class_idx
    else:
      return img, class_idx

### Let's define transform for our data.

In [ ]:
!pip install astropy &> /dev/null
!pip install photutils &> /dev/null

In [ ]:
from photutils.background import Background2D, MedianBackground
import cv2
from astropy.convolution import convolve
from photutils.segmentation import make_2dgaussian_kernel
from photutils.segmentation import detect_sources
from photutils.segmentation import deblend_sources

class CreateSegmap(object):
  def __init__(self, blended:bool=False):
    self.blended = blended

  def __call__(self, original_img):
    #img = np.array(original_img.convert("L")).astype("float64")
    img = original_img.convert("L")
    img = np.array(img)
    img = img.astype("float64")

    #removing background noise
    bkg_estimator = MedianBackground()
    bkg = Background2D(img, (40, 40), filter_size=(7, 7), #The box size should generally be larger than the typical size of sources
                                                          #in the image, but small enough to encapsulate any background variations. For best results, the box size should also
                                                          # be chosen so that the data are covered by an integer number of boxes in both dimensions.
                      bkg_estimator=bkg_estimator, exclude_percentile=20.0)
    img -= bkg.background  # subtract the background

    #Convolvong with 2D kernel
    threshold = 0.5 * bkg.background_rms
    kernel = make_2dgaussian_kernel(3.0, size=5)  # FWHM = 3.0
    convolved_data = convolve(img, kernel)

    #Creating segmap
    segment_map = detect_sources(convolved_data, threshold, npixels=10)

    #Deblending sources from segmap if deblend==True
    if not self.blended:
      candidates = self.find_candidates(segment_map)
      if len(candidates) != 0:
        mask = np.array(segment_map.data == segment_map.labels[candidates[0]])
      else:
        mask = np.ones((segment_map.data.shape[0], segment_map.data.shape[1]))
    else:
      segm_deblend = deblend_sources(convolved_data, segment_map,
                                npixels=10, nlevels=32, contrast=0.001,
                                progress_bar=False)
      candidates = self.find_candidates(segm_deblend)
      if len(candidates) != 0:
        mask = np.array(segm_deblend.data == segm_deblend.labels[candidates[0]])
      else:
        mask = np.ones((segm_deblend.data.shape[0], segm_deblend.data.shape[1]))

    segmented_image = original_img*np.repeat(mask[:, :, np.newaxis], 3, axis=2)#casting 2D grayscale mask to 3D RGB mask repeating it 3 times, can check like [:,:,0], [:,:,1], [:,:,2]
    return Image.fromarray(np.uint8(segmented_image))

  def find_candidates(self, map):
    center = np.array((map.data.shape[0] // 2, map.data.shape[1] // 2))
    candidates = []
    for n, slc in enumerate(map.slices):
      x, y = slc[0], slc[1]
      slc_center = np.array([(x.start + x.stop)//2, (y.start + y.stop)//2])
      metric = ((slc_center[0] - center[0])**2 + (slc_center[1] - center[1])**2)**(1/2)
      if metric < 0.05*map.data.shape[1]:
        candidates.append(n)

    return candidates

  def __repr__(self):
    return "Creates segmentation map, cropping central source"

In [ ]:
'''
For this purpose we should know the minimal size of images provided to rescale them all to it.
'''
image_path_list = list(image_path.glob("*/*/*.jpg"))
min_size = 1000
for image_path in image_path_list:
  img = Image.open(image_path)
  if img.height < min_size:
    min_size = img.height
  #if img.height != img.width: print("Not a square!") all the images are squares
print(f"Minimal size is {min_size}")

In [ ]:
from torchvision import transforms

#So, cause minimal size is 80x80, we should resize all the images to that size

augment_transforms = transforms.Compose([
  #CreateSegmap(blended=False),
  transforms.Resize((80, 80)),#More resolution is not suitable cause minimal size on train set is 80x80
  transforms.RandomHorizontalFlip(p=0.5),
  transforms.RandomVerticalFlip(p=0.5),
  transforms.ToTensor()
])

general_transforms = transforms.Compose([
    CreateSegmap(blended=False), #Segmantation stabilising training process and loss function steps, also helps in generalisation?
    transforms.Resize((80, 80), interpolation=transforms.InterpolationMode.BICUBIC),#More resolution is not suitable cause minimal size on train set is 80x80
    transforms.ToTensor(),
    #transforms.Grayscale()
])

### Custom Transform visualisation

In [ ]:
import random

#random.seed(42)

image_path_list = list(test_dir.glob("*/*.jpg"))

'''
Let's visualise random image
'''
random_image_path = random.choice(image_path_list)
image_class = random_image_path.parent.stem
original_img = Image.open(random_image_path)
original_img

In [ ]:
import matplotlib.pyplot as plt
segmap = CreateSegmap(blended=False)
plt.imshow(segmap(original_img))

### Creating Datasets from our data

In [ ]:
#train_data = CustomDataset(targ_dir=train_dir, transform=augment_transforms)
train_data = CustomDataset(targ_dir=train_dir, transform=general_transforms)
valid_data = CustomDataset(targ_dir=valid_dir, transform=general_transforms)
test_data = CustomDataset(targ_dir=test_dir, transform=general_transforms)

In [ ]:
len(train_data),len(valid_data), len(test_data), train_data.classes, valid_data.classes, test_data.classes, train_data.class_to_idx, len(train_data) + len(valid_data) + len(test_data)

### Creating Dataloaders

In [ ]:
import os

BATCH_SIZE = 16

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              num_workers=torch.cuda.device_count()*2,
                              shuffle=True,
                              pin_memory=True) #should always be True for training on GPU, speeds up the transfer to it

valid_dataloader = DataLoader(dataset=valid_data,
                              batch_size=1,
                              num_workers=torch.cuda.device_count()*2,
                              shuffle=False,
                              pin_memory=True)

test_dataloader = DataLoader(dataset=test_data,
                            batch_size=1,
                            num_workers=torch.cuda.device_count()*2,
                            shuffle=False,
                            pin_memory=True)


In [ ]:
img, label = next(iter(train_dataloader))

print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

### Visualising tensors from dataset

In [ ]:
def display_random_images(dataset: torch.utils.data.dataset.Dataset,
                          classes: List[str] = None,
                          n: int = 10,
                          display_shape: bool = True,
                          seed: int = None):

  import matplotlib.pyplot as plt

  if seed:
    random.seed(seed)

  random_samples_idx = random.sample(range(len(dataset)), k=n)
  plt.figure(figsize=(16, 8))

  for i, targ_sample in enumerate(random_samples_idx):
    targ_image, targ_label = dataset[targ_sample][0], dataset[targ_sample][1]

    # Adjust image tensor shape for plotting: [color_channels, height, width] -> [color_channels, height, width]
    targ_image_adjust = targ_image.permute(1, 2, 0)
    plt.subplot(1, n, i+1)
    plt.imshow(targ_image_adjust)
    plt.axis("off")
    if classes:
      title = f"class: {classes[targ_label]}"
      if display_shape:
        title = title + f"\nshape: {targ_image_adjust.shape}"
    plt.title(title)

In [ ]:
display_random_images(test_data, n=5, classes=test_data.classes, seed=None)

## Part 2 - Building the CNN

In [ ]:
class DoubleConv(nn.Module):
  '''
  Main bulding block of the net. Consists of two consecutive convolutional
  2D layers with ReLU activation and dp% dropout.
  '''
  def __init__(self, in_channels, out_channels, dp):
    super().__init__()
    self.conv = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
      nn.ReLU(),
      nn.Dropout(p=dp),
      nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),#, bias=False), -> if Normalisation exists
      nn.ReLU(),
      nn.Dropout(p=dp),
      nn.MaxPool2d(kernel_size = 2, stride = 2) #The pooling layer summarises the features
                                                #present in a region of the feature map generated by a
                                                #convolution layer. So, further operations are performed on
                                                #summarised features instead of precisely positioned features generated by the
                                                #convolution layer. This makes the model more robust to
                                                #variations in the position of the features in the input image.
    )

  def forward(self, x:torch.Tensor) -> torch.Tensor:
    return self.conv(x)

class PRGClassificator(nn.Module):
  def __init__(self, input_features: int, output_features: int, hidden_units: int, num_blocks: int, dp: float):
    super().__init__()
    self.forward_path_list = nn.ModuleList()

    for block in range(num_blocks):
      self.forward_path_list.append(DoubleConv(input_features, hidden_units, dp))
      input_features = hidden_units

    self.classifier = nn.Sequential(
      nn.Flatten(),
      nn.Linear(int(hidden_units*((80*(1/2**num_blocks))**2)), output_features),#As image size is decreased to NxN
                                                                        #due to num_blocks pooling layers,
                                                                        #where N = 80*(1/2)**(num_blocks)
      nn.Sigmoid()
    )

  def forward(self, x):
    for block in self.forward_path_list:
      x = block(x)
    return self.classifier(x)

### Model analytics

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
#Model initialisation
torch.manual_seed(42)
model = PRGClassificator(input_features=3, # number of color channels
                  output_features=len(train_data.classes),
                  hidden_units=10,
                  num_blocks = 1,
                  dp = 0.5).to(device)

summary(model, input_size=[16, 3, 80, 80]) # do a test pass through of an example input size

In [ ]:
!pip install torchview &> /dev/null

In [ ]:
import graphviz
graphviz.set_jupyter_format('svg')

In [ ]:
from torchview import draw_graph

# device='meta' -> no memory is consumed for visualization
#model_graph = draw_graph(model, input_size=(900, input_features), device='meta', save_graph=True)
model_graph = draw_graph(model, input_size=[16, 3, 80, 80], device='meta', save_graph=True)
model_graph.visual_graph

## Part 3 - Training the CNN

### Defining train_step function

In [ ]:
def train_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              optimizer: torch.optim.Optimizer):

  #import time

  model.train()
  train_loss, train_acc = 0, 0
  #start_time = 0
  for X, y in dataloader:
    X, y = X.to(device), y.to(device)

    # FORWARD PASS
    '''
    For multiclass

    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)
    '''

    # FORWARD PASS
    y_pred = model(X).view(y.shape)
    loss = loss_fn(y_pred, y.float())
    train_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    y_pred_class = y_pred.round()
    train_acc += ((y_pred_class == y).sum().item()/len(y_pred_class))

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

### Defining valid_step function

In [ ]:
def valid_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):

  model.eval()
  valid_loss, valid_acc = 0, 0

  with torch.inference_mode():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)

      '''
      Multiclass
      # FORWARD PASS
      valid_pred_logits = model(X)
      loss = loss_fn(valid_pred_logits, y)
      valid_loss += loss.item()
      valid_pred_labels = valid_pred_logits.argmax(dim=1)
      valid_acc += ((valid_pred_labels == y).sum().item()/len(valid_pred_labels))
      '''
      # FORWARD PASS
      valid_pred_logits = model(X).view(y.shape)
      loss = loss_fn(valid_pred_logits, y.float())
      valid_loss += loss.item()
      valid_pred_class = valid_pred_logits.round()
      valid_acc += ((valid_pred_class == y).sum().item()/len(valid_pred_class))

  # Adjust metrics to get average loss and accuracy per batch
  valid_loss = valid_loss / len(dataloader)
  valid_acc = valid_acc / len(dataloader)
  return valid_loss, valid_acc

### Defining whole training function

In [ ]:
def train(model_name: str,
          model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          valid_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 20):

  min_val_acc = 0.6
  torch.manual_seed(42)

  from tqdm.auto import tqdm

  results = {"model_name": [],
    "train_loss": [],
    "train_acc": [],
    "valid_loss": [],
    "valid_acc": []
  }
  print(model_name)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                        dataloader=train_dataloader,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer)
    valid_loss, valid_acc = valid_step(model=model,
        dataloader=valid_dataloader,
        loss_fn=loss_fn)

    scheduler.step(valid_loss)

    print(
      f"Epoch: {epoch+1} | "
      f"train_loss: {train_loss:.4f} | "
      f"train_acc: {train_acc:.4f} | "
      f"valid_loss: {valid_loss:.4f} | "
      f"valid_acc: {valid_acc:.4f}"
    )

    if valid_acc >= min_val_acc:
      torch.save(model.state_dict(), model_name)
      print(f"Saved accuracy is {valid_acc}")
      min_val_acc = valid_acc

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["valid_loss"].append(valid_loss)
    results["valid_acc"].append(valid_acc)

  results["model_name"].append(model_name)
  return results

### Training Loop

In [ ]:
num_blocks = [1]#[1, 2, 3]#4 blocks tested, stable 50%
hidden_units = [10]#[10, 50, 100]#[10, 50, 100, 200, 400]
learning_rates = [1e-4]#[1e-3, 1e-4]#[1e-3, 5e-4, 1e-4, 5e-5]
dropouts = [0.5]#[0.1, 0.2]
hyperparams_grid = []
for block in num_blocks:
  for hu in hidden_units:
    for lr in learning_rates:
      for dropout in dropouts:
        hyperparams_grid.append((block, hu, lr, dropout))

In [ ]:
'''
CHECK THESE

For ['model_hu10_blocks1_lr0.001'] best valid_loss is 0.6476480677723885 for acc 0.75
For ['model_hu10_blocks1_lr0.0005'] best valid_loss is 0.5645014506648295 for acc 0.7     -
For ['model_hu50_blocks1_lr0.001'] best valid_loss is 0.6493203544799598 for acc 0.75
For ['model_hu100_blocks1_lr0.001'] best valid_loss is 0.6723072484135628 for acc 0.75
For ['model_hu100_blocks1_lr0.0005'] best valid_loss is 0.4840030772611385 for acc 0.7    -
For ['model_hu200_blocks1_lr0.0005'] best valid_loss is 0.667924758605659 for acc 0.75
For ['model_hu200_blocks1_lr0.0001'] best valid_loss is 0.6068458913825452 for acc 0.7
For ['model_hu200_blocks1_lr5e-05'] best valid_loss is 0.6206109322607517 for acc 0.75    -
For ['model_hu400_blocks1_lr0.0005'] best valid_loss is 0.6931484043598175 for acc 0.7    -
For ['model_hu10_blocks2_lr0.0005'] best valid_loss is 0.6179103586357086 for acc 0.7     -
For ['model_hu200_blocks2_lr0.0001'] best valid_loss is 0.6879961729049683 for acc 0.75
For ['model_hu400_blocks2_lr0.0001'] best valid_loss is 0.6551367722451686 for acc 0.7    -
For ['model_hu100_blocks3_lr0.0005'] best valid_loss is 0.6941133096814156 for acc 0.75

BEST: model_hu10_blocks1_lr0.0001_dp0.25, model_hu50_blocks1_lr0.0001_dp0.2
'''

hyperparams_grid = [(1, 50, 0.0001, 0.1)]#[(1, 50, 0.0001, 0.2)]

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

NUM_EPOCHS = 50

grid_results = []
for params in hyperparams_grid:
  model_results = {}
  num_blocks = params[0]
  hidden_units = params[1]
  lr = params[2]
  dropout = params[3]
  #Model initialisation
  model = PRGClassificator(input_features=3, # number of color channels
                          hidden_units=hidden_units,
                          output_features=1,#len(train_data.classes),
                          num_blocks = num_blocks,
                          dp = dropout).to(device)

  # Choosing Loss function and optimizer
  #loss_fn = nn.CrossEntropyLoss()
  loss_fn = nn.BCELoss()
  optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr, weight_decay=1e-3)#wd == L2 regularization

  # Training model
  model_results = train(model_name = f"model_hu{hidden_units}_blocks{num_blocks}_lr{lr}_dp{dropout}",
                        model=model,
                        train_dataloader=train_dataloader,
                        valid_dataloader=valid_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS)
  grid_results.append(model_results)

In [ ]:
for result in grid_results:
  if max(result['valid_acc']) >= 0.7:
    print(f"For {result['model_name']} best valid_loss is {min(result['valid_loss'])} for acc {max(result['valid_acc'])}")

### Analysing results

In [ ]:
import matplotlib.pyplot as plt

loss = model_results['train_loss']
valid_loss = model_results['valid_loss']
accuracy = model_results['train_acc']
valid_accuracy = model_results['valid_acc']
epochs = range(len(model_results['train_loss']))

# Setup a plot
plt.figure(figsize=(15, 7))

plt.subplot(1, 2, 1)
plt.plot(epochs, loss, label='train_loss')
plt.plot(epochs, valid_loss, label='validation_loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, accuracy, label='train_accuracy')
plt.plot(epochs, valid_accuracy, label='validation_accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.legend()

In [ ]:
print(loss[90:], valid_loss[90:])
print(accuracy[90:], valid_accuracy[90:])

## Part 4 - Making predictions on test set

In [ ]:
test_loss, test_acc = 0, 0

model = PRGClassificator(input_features=3, # number of color channels
                          hidden_units=10,
                          output_features=1,
                          num_blocks = 1,
                          dp = 0.5).to(device)
model.load_state_dict(torch.load("model_hu10_blocks1_lr0.0001_dp0.5", map_location=torch.device('cpu')))
model.eval()
test_labels, pred_labels = [], []
with torch.inference_mode():
  fp = 0
  for batch, (X, y) in enumerate(test_dataloader):
    X, y = X.to(device), y.to(device)

    test_pred_logits = model(X)
    loss = loss_fn(test_pred_logits.view(y.shape), y.float())
    test_loss += loss.item()

    test_pred_labels = test_pred_logits.round()
    if test_pred_labels != y:
      #print(f"Model named {test_data.classes[y]} galaxy as {test_data.classes[test_pred_labels]} galaxy!")
      if y == 0:
        fp += 1
        print(f"Galaxy classified as prg is {test_data.paths[batch]}")
    test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
    test_labels.append(y.item())
    pred_labels.append(test_pred_labels.item())

# Adjust metrics to get average loss and accuracy per batch
test_loss = test_loss / len(test_dataloader)
test_acc = test_acc / len(test_dataloader)

print(test_loss, test_acc, fp)

In [ ]:
!pip install torchmetrics &> /dev/null
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
import matplotlib.pyplot as plt
confmat = ConfusionMatrix(num_classes=len(test_data.classes), task='multiclass')
confmat_tensor = confmat(preds = torch.Tensor(pred_labels), target = torch.Tensor(test_labels))

fig, ax = plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(),
    class_names=list(test_data.class_to_idx.keys()),
    figsize=(10, 7)
)

plt.savefig("conf_matrix.pdf", dpi=400, format='pdf', bbox_inches='tight')